In [1]:
!pip install -q ragstack-ai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import cassio

In [3]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] ="ASTRA_DB_API_ENDPOINT"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] ="ASTRA_DB_APPLICATION_TOKEN"

**Create RAG Pipeline**

In [5]:
os.environ["GOOGLE_API_KEY"]="GOOGLE_API_KEY"

In [7]:
from langchain_astradb import AstraDBVectorStore
from langchain_community.embeddings.google_palm import GooglePalmEmbeddings
import os

# Configure your embedding model and vector store

vstore = AstraDBVectorStore(
    collection_name="test",
    embedding=GooglePalmEmbeddings(),
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

Astra vector store configured


In [8]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/450 [00:00<?, ? examples/s]

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [9]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [10]:
docs

[Document(metadata={'author': 'aristotle', 'knowledge': 'y'}, page_content="True happiness comes from gaining insight and growing into your best possible self. Otherwise all you're having is immediate gratification pleasure, which is fleeting and doesn't grow you as a person."),
 Document(metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}, page_content='The roots of education are bitter, but the fruit is sweet.'),
 Document(metadata={'author': 'aristotle', 'ethics': 'y'}, page_content='Before you heal the body you must first heal the mind'),
 Document(metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}, page_content='The proof that you know something is that you are able to teach it'),
 Document(metadata={'author': 'aristotle'}, page_content='Those who are not angry at the things they should be angry at are thought to be fools, and so are those who are not angry in the right way, at the right time, or with the right persons.'),
 Document(metadata={'au

In [11]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [12]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': '4b54625918d14194a4efc06b2005dd29', 'content': 'Criticism alone can sever the root of materialism, fatalism, atheism, free-thinking, fanaticism, and superstition, which can be injurious universally; as well as of idealism and skepticism, which are dangerous chiefly to the Schools, and hardly allow of being handed on to the public.', 'metadata': {'author': 'kant', 'education': 'y'}}, {'_id': '543595d89c734de9b17a5c540cb4854c', 'content': 'The State is the Divine idea as it exists on Earth.', 'metadata': {'author': 'hegel'}}, {'_id': 'cf2b6291b5714f3fbdd0a3cd76736648', 'content': 'Mankind will never see an end of trouble until lovers of wisdom come to hold political power, or the holders of power become lovers of wisdom', 'metadata': {'author': 'plato', 'politics': 'y', 'knowledge': 'y'}}, {'_id': 'c000265c8bf9488c822bf7277f8646c9', 'content': 'The more intelligible a thing is, the more easily it is retained in the memory, and counterwise, the less intelli

In [15]:
!pip install langchain-groq groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.22
    Uninstalling langchain-core-0.2.22:
      Successfully uninstalled langchain-core-0.2.22
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragstack-ai-langchain 1.4.0 requires langchain-core==0.2.22, but you have langchain-core 0.2.32 which is incompatible.


In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.google_palm import GooglePalm
from langchain_groq import ChatGroq
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})
GROQ_API_KEY="<GROQ_API_KEY>"

model =ChatGroq(groq_api_key=GROQ_API_KEY,
             model_name="mixtral-8x7b-32768")
prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
# model = GooglePalm(os.getenv("GOOGLE_API_KEY"),temperature=0.1)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what is the most important to allow the brain and provide me the tags?")

"Based on the context provided, the most important aspect for intellectual development involves training the brain to be patient, to defer judgment, and to approach and grasp individual cases from all sides. This is because it allows for the establishment of true connections between different pieces of information or perception, and prevents the brain from fabricating false connections when it is unable to establish a true one. The metadata tags associated with this answer would be 'patience', 'delayed judgment', and 'holistic approach'."

In [17]:
chain.invoke("what is the important aspect?")

'Based on the supplied context, the important aspects mentioned are:\n\n1. In the document by Hegel, the importance is placed on the modern state being organized and articulated, which occurs when the universal is bound up with the complete freedom of its particular members and their private well-being, and when the interests of family and civil society concentrate themselves on the state.\n\n2. In the document by Hegel, the importance of interest or passion is emphasized as a driving force behind great accomplishments, where the whole individuality is devoted to an object with every fibre of volition, concentrating all its desires and powers upon it.\n\n3. In the document by Nietzsche, the importance is given to setting high and noble goals for oneself and perishing in pursuit of them, rather than being concerned with the purpose of humanity or having a ready answer for why one is here.\n\nThese are the aspects that are highlighted in the given documents as important.'

In [ ]:
# WARNING: This will delete the collection and all documents in the collection
vstore.delete_collection()